# Using `Crystals.jl`

Content of this notebook:
1. Create [fcc/bcc](https://en.wikipedia.org/wiki/Cubic_crystal_system) unit cells
2. Use the unit cell to create supercells
3. "Add" a vacancy to the supercell

In [ ]:
import Pkg

In [ ]:
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
Pkg.status()

In [ ]:
using Crystal
using DataFrames
using LaTeXStrings
using Plots

## Creating Crystals

Let's first define some constants

In [ ]:
masses = Dict("V" => 50.9415, "Nb" => 92.9064, "Ta" => 180.9479,
              "Cr" => 51.996, "Mo" => 95.94, "W" => 183.85,
              "Fe" => 55.847)
el2atom_map = Dict(el => Crystal.Atom(name=el, mass=masses[el]) for el in keys(masses))

# Å
bcc_lattice_constants = Dict(
    "V" => 3.0399, "Nb" => 3.3008, 
    "Ta" => 3.3058, "Cr" => 2.8845, "Mo" => 3.1472, 
    "W" => 3.1652, "Fe" => 2.8665
)

## Synthesizing crystal unit cells

Crystals are something fascinating. Defect free crystals are highly symmetric and can be reduced to so-called "unit cells", a cell which can be used by copying and shifting it to construct the entire crystal. So to sound impressive the crystal of multiple unit cells is called a supercell ¯\\\_(ツ)\_/¯.

So in a first step we'll define how to create two common types of unit cells and then go on to synthesize a supercell.

### body centered cubic crystal

In [ ]:
el = "Fe"
a = bcc_lattice_constants[el]
elements = [el for _ in 1:2]
cell = Crystal.make_bcc_unitcell(elements, a, el2atom_map)

Visualizing the crystal

In [ ]:
function plot_crystal(cell::Crystal.Cell;
        default_color::String="blue",
        element_color_map::Dict=Dict{String,String}("Fe" => "blue"),
        default_size::T=50,
        element_size_map::Dict=Dict{String,Any}()
    ) where T <: Real
    
    atoms, coords = cell.atoms, cell.coords 
    
    elements = Set([atom.name for atom in atoms])
    for element in elements
        if !haskey(element_color_map, element)
            element_color_map[element] = default_color
        end
        if !haskey(element_size_map, element)
            element_size_map[element] = default_size
        end
    end
    colors = [element_color_map[atom.name] for atom in atoms]
    sizes = [element_size_map[atom.name] for atom in atoms]

    x = [v[1] for v in coords]
    y = [v[2] for v in coords]
    z = [v[3] for v in coords]
    return @gif for i in range(0, stop=2π, length=100)
        scatter(x, y, z, camera=(10*(1+cos(i)),5),
            markersize=sizes, legend=false, 
            color=colors, aspect_ratio=:equal,
            xlabel=L"x", ylabel=L"y", zlabel=L"z",
            title=string(length(atoms), " atoms of: ", join(elements, ","))
        )
    end
end

In [ ]:
plot_crystal(cell, default_size=50)

### face centered cubic crystal

In [ ]:
el = "Fe"
a = bcc_lattice_constants[el]
elements = [el for _ in 1:4]
cell = Crystal.make_fcc_unitcell(elements, a, el2atom_map);

In [ ]:
plot_crystal(cell, default_size=50)

## Generating a supercell from a unit cell

In [ ]:
el = "Fe"
unitcell = Crystal.make_bcc_unitcell([el for _ in 1:2], bcc_lattice_constants[el], el2atom_map)
supercell = Crystal.make_supercell(unitcell, nx=3, ny=3, nz=3);

In [ ]:
plot_crystal(supercell, default_size=5)

## Inserting a vacancy / removing an atom 

In [ ]:
vac = Crystal.add_vacancies(supercell, ixs=[1]);

In [ ]:
plot_crystal(vac, default_size=5)

Looks okay. Next, let's verify that the distance distributions of the first neighbours make sense.

## Sanity checking distances

### bcc

In [ ]:
struct NeighbourFinder 
    nb_matrix::BitArray{2} # defines which atom pairs we'll be happy to check at all
    dist_cutoff::Float32
    rcut2::Float32
end

NeighbourFinder(nb_matrix, dist_cutoff) = NeighbourFinder(nb_matrix, dist_cutoff, dist_cutoff^2)

function find_neighbours(cell::Crystal.Cell, nf::NeighbourFinder)
    neighbours = []
    rs = []
    for i in 1:length(cell.coords)
        ci = cell.coords[i]
        for j in 1:length(cell.coords)
            if i==j 
                continue
            end
            
            r2 = sum(abs2, Crystal.vector(ci, cell.coords[j], cell.edge_lengths))
            if r2 <= nf.rcut2 && nf.nb_matrix[j,i]
                push!(neighbours, (i,j))
                push!(rs, sqrt(r2))
            end                
        end
    end
    return neighbours, rs
end

function get_distance_df(cell::Crystal.Cell; dist_cutoff::Real=2)
    n_atoms = length(cell.atoms)
    nb_matrix = trues(n_atoms,n_atoms)
    nf = NeighbourFinder(nb_matrix, dist_cutoff)
    idxs, rs = find_neighbours(cell, nf)
    rs_df = sort(combine(groupby(DataFrame("distances"=>rs),[:distances]), 
 nrow=>:count), [:distances])
    return rs_df, rs
end

Defining the supercell

In [ ]:
el = "Fe"
unitcell = Crystal.make_bcc_unitcell([el for _ in 1:2], 1, el2atom_map)
supercell = Crystal.make_supercell(unitcell, nx=3, ny=3, nz=3);

Computing pairwise distances

In [ ]:
d = 2
rs_df, rs = get_distance_df(supercell, dist_cutoff=d)

Visualizing the distance distribution

In [ ]:
function plot_distance_hist(rs::Array, title::String, cutoff)
    histogram(rs, xlabel=L"r", ylabel="Frequency", 
        title=string(title, ": euclidan (periodic) distance distribution (rcut ",cutoff,")"),
        bins=200,
    )
end

In [ ]:
plot_distance_hist(rs, "bcc", d)

### fcc

Same steps but for fcc

In [ ]:
el = "Fe"
unitcell = Crystal.make_fcc_unitcell([el for _ in 1:4], 1, el2atom_map)
supercell = Crystal.make_supercell(unitcell, nx=3, ny=3, nz=3);

In [ ]:
d = 2
rs_df, rs = get_distance_df(supercell, dist_cutoff=d)

In [ ]:
plot_distance_hist(rs, "fcc", d)